<a href="https://colab.research.google.com/github/pnandini-sdu/AAI-511-Final-Project/blob/main/FinalTeam_Project_AAI511_CNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install pretty_midi

In [8]:
import os, glob, itertools, math
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pretty_midi
from collections import defaultdict, Counter
from pathlib import Path

# Mounting content from my google drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

# ======================
# Config (from EDA)
# ======================
DATA_DIR = "/content/drive/My Drive/music_dataset"
CLASSES    = ["Bach", "Beethoven", "Chopin", "Mozart"]
MIDI_EXTS  = [".mid", ".midi"]

FS         = 8            # frames/sec for piano roll
WIN_SECS   = 10           # window length
HOP_SECS   = 5            # hop
WIN        = WIN_SECS * FS
HOP        = HOP_SECS * FS

PITCH_LOW  = 25           # crop inclusive
PITCH_HIGH = 100          # crop inclusive
PITCHES    = PITCH_HIGH - PITCH_LOW + 1  # = 75

BATCH_SIZE = 16
EPOCHS     = 25
LR         = 4e-4
SEED       = 1337

rng = np.random.default_rng(SEED)
tf.keras.utils.set_random_seed(SEED)


def iter_midi_files(root: Path):
    # Recursively yield all .mid/.midi (case‑insensitive), skipping hidden dirs
    for p in root.rglob("*"):
        if p.is_file() and p.suffix.lower() in MIDI_EXTS and not any(part.startswith(".") for part in p.parts):
            yield str(p)

def list_files_recursive(data_dir, classes):
    # Ensure it's a Path object
    data_dir = Path(data_dir)

    files, labels = [], []
    for idx, comp in enumerate(classes):
        comp_dir = data_dir / comp
        if not comp_dir.exists():
            print(f"[warn] missing composer dir: {comp_dir}")
            continue

        for file_path in comp_dir.rglob("*"):
            if file_path.suffix.lower() in (".mid", ".midi"):
                files.append(str(file_path))
                labels.append(idx)

    return files, labels

# ======================
# Split (reuse or create)
# ======================


files, labels = list_files_recursive(DATA_DIR, CLASSES)

# Stratified piece-level split (feel free to replace with your saved LSTM split)
def make_split(files, labels, val_ratio=0.15, test_ratio=0.15):
    by_class = defaultdict(list)
    for p, y in zip(files, labels):
        by_class[y].append(p)
    train, val, test = [], [], []
    for y, items in by_class.items():
        rng.shuffle(items)
        n=len(items); n_test=round(n*test_ratio); n_val=round(n*val_ratio)
        test.extend((p,y) for p in items[:n_test])
        val .extend((p,y) for p in items[n_test:n_test+n_val])
        train.extend((p,y) for p in items[n_test+n_val:])
    rng.shuffle(train); rng.shuffle(val); rng.shuffle(test)
    return train, val, test

train, val, test = make_split(files, labels)
print(f"Pieces -> train: {len(train)}, val: {len(val)}, test: {len(test)}")

# ======================
# MIDI -> Piano-roll utils
# ======================
def midi_to_roll(path, fs=FS):
    """Return (128, T) binary piano-roll; robust to odd files."""
    try:
        pm = pretty_midi.PrettyMIDI(path)
        T = int(np.ceil(pm.get_end_time() * fs)) + 1
        roll = np.zeros((128, T), dtype=np.uint8)
        for inst in pm.instruments:
            for note in inst.notes:
                s = max(0, int(np.floor(note.start * fs)))
                e = min(T, int(np.ceil(note.end * fs)))
                if e > s:
                    roll[note.pitch, s:e] = 1
        return roll
    except Exception as e:
        print(f"[warn] failed {path}: {e}")
        return None

def cap_sustain(roll, max_secs=4.0, fs=FS):
    """Cap continuous '1' runs per pitch to reduce pedal-induced blobs."""
    max_len = int(max_secs * fs)
    R = roll.copy()
    for p in range(R.shape[0]):
        row = R[p]
        i = 0
        while i < row.size:
            if row[i] == 1:
                j = i
                while j < row.size and row[j] == 1:
                    j += 1
                if (j - i) > max_len:
                    row[i+max_len:j] = 0
                i = j
            else:
                i += 1
    return R

def crop_roll(roll, low=PITCH_LOW, high=PITCH_HIGH):
    return roll[low:high+1]

def window_roll(roll, win=WIN, hop=HOP):
    T = roll.shape[1]
    if T < win:
        pad = np.zeros((roll.shape[0], win - T), dtype=roll.dtype)
        return [np.concatenate([roll, pad], axis=1)]
    return [roll[:, s:s+win] for s in range(0, T - win + 1, hop)]

# ======================
# Augmentation (conservative from EDA)
# ======================
def augment_roll(roll, max_semitones=2, time_scales=(0.9, 1.0, 1.1)):
    # pitch shift
    st = rng.integers(-max_semitones, max_semitones+1)
    aug = roll
    if st != 0:
        aug = np.roll(aug, shift=st, axis=0)
        if st > 0:  aug[:st, :] = 0
        else:       aug[st:, :] = 0
    # time stretch
    scale = rng.choice(time_scales)
    if scale != 1.0:
        T = aug.shape[1]
        new_T = int(round(T * scale))
        idx = np.clip((np.arange(new_T) / scale).astype(int), 0, T-1)
        aug = aug[:, idx]
    return aug

def to_input(chunk):
    x = chunk.astype(np.float32)  # keep density info
    return x[..., None]           # (H, W, 1)

# ======================
# tf.data pipeline
# ======================
def gen_examples(pairs, augment=False):
    for path, y in pairs:
        roll = midi_to_roll(path)
        if roll is None:
            continue
        roll = cap_sustain(roll, max_secs=4.0, fs=FS)
        roll = crop_roll(roll)
        for ch in window_roll(roll):
            if augment:
                ch = augment_roll(ch)
                # re-fit to exact window after time-stretch
                if ch.shape[1] < WIN:
                    pad = np.zeros((PITCHES, WIN - ch.shape[1]), dtype=ch.dtype)
                    ch = np.concatenate([ch, pad], axis=1)
                elif ch.shape[1] > WIN:
                    ch = ch[:, :WIN]
            yield to_input(ch), y

def make_dataset(pairs, shuffle=True, augment=False):
    spec = (
        tf.TensorSpec(shape=(PITCHES, WIN, 1), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
    ds = tf.data.Dataset.from_generator(lambda: gen_examples(pairs, augment=augment),
                                        output_signature=spec)
    if shuffle:
        ds = ds.shuffle(8192, seed=SEED, reshuffle_each_iteration=True)
    return ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

train_ds = make_dataset(train, shuffle=True,  augment=True)
val_ds   = make_dataset(val,   shuffle=False, augment=False)
test_ds  = make_dataset(test,  shuffle=False, augment=False)

# ======================
# Class weights from WINDOW COUNTS (post-slicing)
# ======================
def count_windows(pairs):
    counts = Counter()
    for path, y in pairs:
        roll = midi_to_roll(path)
        if roll is None:
            continue
        n = len(window_roll(crop_roll(cap_sustain(roll))))
        counts[y] += n
    return counts

win_counts = count_windows(train)
total = sum(win_counts.values())
class_weight = {}
for k in range(len(CLASSES)):
    # inverse frequency
    freq = win_counts.get(k, 1)
    class_weight[k] = total / (len(CLASSES) * freq)
print("Window counts per class:", {CLASSES[k]: v for k, v in win_counts.items()})
print("Class weights:", {CLASSES[k]: round(v, 3) for k, v in class_weight.items()})

# ======================
# CNN (EDA-informed: mixed kernels, dropout, BN)
# ======================
def build_cnn(input_shape=(PITCHES, WIN, 1), num_classes=len(CLASSES)):
    inp = keras.layers.Input(shape=input_shape)
    x = inp

    # Block 1: small textures
    x = keras.layers.Conv2D(32, (3,3), padding="same", activation="relu")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv2D(32, (3,3), padding="same", activation="relu")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D((2,2))(x)         # H/2, W/2
    x = keras.layers.Dropout(0.2)(x)

    # Block 2: widen features
    x = keras.layers.Conv2D(64, (3,3), padding="same", activation="relu")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv2D(64, (5,3), padding="same", activation="relu")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D((2,2))(x)         # H/4, W/4
    x = keras.layers.Dropout(0.25)(x)

    # Block 3: capture large leaps / thick chords
    x = keras.layers.Conv2D(128, (7,3), padding="same", activation="relu")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Conv2D(128, (3,3), padding="same", activation="relu")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.MaxPooling2D((2,2))(x)         # H/8, W/8
    x = keras.layers.Dropout(0.3)(x)

    # Head
    x = keras.layers.Conv2D(256, (3,3), padding="same", activation="relu")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(128, activation="relu")(x)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Dropout(0.35)(x)
    out = keras.layers.Dense(num_classes, activation="softmax")(x)

    model = keras.Model(inp, out)
    model.compile(optimizer=tf.keras.optimizers.Adam(LR),
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    return model

model = build_cnn()
model.summary()

# ======================
# Train
# ======================
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_accuracy", factor=0.5, patience=4, min_lr=1e-5, verbose=1),
    tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=8, restore_best_weights=True, verbose=1)
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    class_weight=class_weight,
    callbacks=callbacks
)


Mounted at /content/drive
Pieces -> train: 1140, val: 245, test: 245
Window counts per class: {'Beethoven': 15806, 'Mozart': 13972, 'Bach': 21660, 'Chopin': 4183}
Class weights: {'Bach': 0.642, 'Beethoven': 0.88, 'Chopin': 3.324, 'Mozart': 0.995}


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 76, 80, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_28 (Conv2D)              │ (None, 76, 80, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_32          │ (None, 76, 80, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 76, 80, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_33          │ (None, 76, 80, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 38, 40, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 38, 40, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_30 (Conv2D)              │ (None, 38, 40, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_34          │ (None, 38, 40, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_31 (Conv2D)              │ (None, 38, 40, 64)     │        61,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_35          │ (None, 38, 40, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 19, 20, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 19, 20, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_32 (Conv2D)              │ (None, 19, 20, 128)    │       172,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_36          │ (None, 19, 20, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_33 (Conv2D)              │ (None, 19, 20, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_37          │ (None, 19, 20, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 9, 10, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 9, 10, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_34 (Conv2D)              │ (None, 9, 10, 256)     │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_38          │ (None, 9, 10, 256)     │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 741,220 (2.83 MB)

 Trainable params: 739,556 (2.82 MB)

 Non-trainable params: 1,664 (6.50 KB)

Epoch 1/25
   3477/Unknown 199s 46ms/step - accuracy: 0.5158 - loss: 1.2058[warn] failed /content/drive/My Drive/music_dataset/Beethoven/Anhang 14-3.mid: Could not decode key with 3 flats and mode 255
3477/3477 ━━━━━━━━━━━━━━━━━━━━ 234s 56ms/step - accuracy: 0.5158 - loss: 1.2057 - val_accuracy: 0.5663 - val_loss: 1.1825 - learning_rate: 4.0000e-04
Epoch 2/25
3473/3477 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.6723 - loss: 0.7766[warn] failed /content/drive/My Drive/music_dataset/Beethoven/Anhang 14-3.mid: Could not decode key with 3 flats and mode 255
3477/3477 ━━━━━━━━━━━━━━━━━━━━ 213s 54ms/step - accuracy: 0.6723 - loss: 0.7765 - val_accuracy: 0.6956 - val_loss: 0.7530 - learning_rate: 4.0000e-04
Epoch 3/25
3468/3477 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.7140 - loss: 0.6800[warn] failed /content/drive/My Drive/music_dataset/Beethoven/Anhang 14-3.mid: Could not decode key with 3 flats and mode 255
3477/3477 ━━━━━━━━━━━━━━━━━━━━ 213s 54ms/step - accuracy: 0.7140 - loss

**Model tunning to improve accuracy**

In [9]:
#Imporing all necessary libraries
import os, glob, itertools, math
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pretty_midi
from collections import defaultdict, Counter
from pathlib import Path

# Mounting content from my google drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

# ======================
# Config (from EDA)
# ======================
DATA_DIR = "/content/drive/My Drive/music_dataset"
CLASSES    = ["Bach", "Beethoven", "Chopin", "Mozart"]
MIDI_EXTS  = [".mid", ".midi"]

FS         = 8            # frames/sec for piano roll
WIN_SECS   = 10           # window length
HOP_SECS   = 5            # hop
WIN        = WIN_SECS * FS
HOP        = HOP_SECS * FS

PITCH_LOW  = 25           # crop inclusive
PITCH_HIGH = 100          # crop inclusive
PITCHES    = PITCH_HIGH - PITCH_LOW + 1  # = 75

BATCH_SIZE = 16
EPOCHS     = 25
LR         = 4e-4
SEED       = 1337

rng = np.random.default_rng(SEED)
tf.keras.utils.set_random_seed(SEED)


def iter_midi_files(root: Path):
    # Recursively yield all .mid/.midi (case‑insensitive), skipping hidden dirs
    for p in root.rglob("*"):
        if p.is_file() and p.suffix.lower() in MIDI_EXTS and not any(part.startswith(".") for part in p.parts):
            yield str(p)

def list_files_recursive(data_dir, classes):
    # Ensure it's a Path object
    data_dir = Path(data_dir)

    files, labels = [], []
    for idx, comp in enumerate(classes):
        comp_dir = data_dir / comp
        if not comp_dir.exists():
            print(f"[warn] missing composer dir: {comp_dir}")
            continue

        for file_path in comp_dir.rglob("*"):
            if file_path.suffix.lower() in (".mid", ".midi"):
                files.append(str(file_path))
                labels.append(idx)

    return files, labels


# ======================
# Split (reuse or create)
# ======================


files, labels = list_files_recursive(DATA_DIR, CLASSES)

# Stratified piece-level split (feel free to replace with your saved LSTM split)
def make_split(files, labels, val_ratio=0.15, test_ratio=0.15):
    by_class = defaultdict(list)
    for p, y in zip(files, labels):
        by_class[y].append(p)
    train, val, test = [], [], []
    for y, items in by_class.items():
        rng.shuffle(items)
        n=len(items); n_test=round(n*test_ratio); n_val=round(n*val_ratio)
        test.extend((p,y) for p in items[:n_test])
        val .extend((p,y) for p in items[n_test:n_test+n_val])
        train.extend((p,y) for p in items[n_test+n_val:])
    rng.shuffle(train); rng.shuffle(val); rng.shuffle(test)
    return train, val, test

train, val, test = make_split(files, labels)
print(f"Pieces -> train: {len(train)}, val: {len(val)}, test: {len(test)}")


# --- Multi-channel MIDI loader ---
def midi_to_roll_multich(path, fs=FS):
    try:
        pm = pretty_midi.PrettyMIDI(path)
        T = int(np.ceil(pm.get_end_time() * fs)) + 1
        on = np.zeros((128, T), dtype=np.float32)
        vel = np.zeros((128, T), dtype=np.float32)
        for inst in pm.instruments:
            for note in inst.notes:
                s = max(0, int(np.floor(note.start * fs)))
                e = min(T, int(np.ceil(note.end * fs)))
                on[note.pitch, s:e] = 1.0
                vel[note.pitch, s:e] = max(vel[note.pitch, s:e].max(), note.velocity / 127.0)
        onset = np.zeros_like(on)
        onset[:, 1:] = np.maximum(0.0, on[:, 1:] - on[:, :-1])
        return on, vel, onset
    except Exception as e:
        print(f"[warn] failed {path}: {e}")
        return None

# --- Multi-channel input stacker ---
def to_input_multich(on, vel, onset):
    return np.stack([on, vel, onset], axis=-1)  # (H, W, 3)

# --- Balanced training dataset generator ---
STEPS_PER_CLASS = 1200  # tune as needed

def balanced_train_ds_multich(pairs):
    per_class = {k: [] for k in range(len(CLASSES))}
    for path, y in pairs:
        per_class[y].append(path)
    for k in per_class:
        rng.shuffle(per_class[k])

    def gen():
        counts = {k: 0 for k in per_class}
        iters = {k: iter(per_class[k]) for k in per_class}
        while min(counts.values()) < STEPS_PER_CLASS:
            for k in per_class:
                if counts[k] >= STEPS_PER_CLASS:
                    continue
                try:
                    p = next(iters[k])
                except StopIteration:
                    continue
                m = midi_to_roll_multich(p)
                if m is None:
                    continue
                on, vel, onset = m
                on = crop_roll(cap_sustain(on))
                vel = crop_roll(cap_sustain(vel))
                onset = crop_roll(cap_sustain(onset))
                on_windows = window_roll(on)
                vel_windows = window_roll(vel)
                onset_windows = window_roll(onset)
                for chunk_on, chunk_vel, chunk_onset in zip(on_windows, vel_windows, onset_windows):
                    yield to_input_multich(chunk_on, chunk_vel, chunk_onset), k
                    counts[k] += 1
                    if counts[k] >= STEPS_PER_CLASS:
                        break

    spec = (tf.TensorSpec((PITCHES, WIN, 3), tf.float32),
            tf.TensorSpec((), tf.int32))
    return tf.data.Dataset.from_generator(gen, output_signature=spec).shuffle(8192).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# --- Standard dataset generator for val/test ---
def gen_examples_multich(pairs, augment=False):
    for path, y in pairs:
        m = midi_to_roll_multich(path)
        if m is None:
            continue
        on, vel, onset = m
        on = cap_sustain(on)
        vel = cap_sustain(vel)
        onset = cap_sustain(onset)

        on = crop_roll(on)
        vel = crop_roll(vel)
        onset = crop_roll(onset)

        on_windows = window_roll(on)
        vel_windows = window_roll(vel)
        onset_windows = window_roll(onset)

        for chunk_on, chunk_vel, chunk_onset in zip(on_windows, vel_windows, onset_windows):
            if augment:
                # Optional augmentation code here (same as before) if you want
                pass
            yield to_input_multich(chunk_on, chunk_vel, chunk_onset), y

def make_dataset_multich(pairs, shuffle=True, augment=False):
    spec = (
        tf.TensorSpec(shape=(PITCHES, WIN, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    )
    ds = tf.data.Dataset.from_generator(lambda: gen_examples_multich(pairs, augment=augment),
                                        output_signature=spec)
    if shuffle:
        ds = ds.shuffle(8192, seed=SEED, reshuffle_each_iteration=True)
    return ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# --- Create datasets ---
train_ds = balanced_train_ds_multich(train)
val_ds   = make_dataset_multich(val, shuffle=False, augment=False)
test_ds  = make_dataset_multich(test, shuffle=False, augment=False)


# --- Build model with updated input shape ---
model = build_cnn(input_shape=(PITCHES, WIN, 3))
model.summary()


# --- Train without class weights ---
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)


Mounted at /content/drive
Pieces -> train: 1140, val: 245, test: 245


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_9 (InputLayer)      │ (None, 76, 80, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_35 (Conv2D)              │ (None, 76, 80, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_40          │ (None, 76, 80, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_36 (Conv2D)              │ (None, 76, 80, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_41          │ (None, 76, 80, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 38, 40, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 38, 40, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_37 (Conv2D)              │ (None, 38, 40, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_42          │ (None, 38, 40, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_38 (Conv2D)              │ (None, 38, 40, 64)     │        61,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_43          │ (None, 38, 40, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 19, 20, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 19, 20, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_39 (Conv2D)              │ (None, 19, 20, 128)    │       172,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_44          │ (None, 19, 20, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_40 (Conv2D)              │ (None, 19, 20, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_45          │ (None, 19, 20, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 9, 10, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_27 (Dropout)            │ (None, 9, 10, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_41 (Conv2D)              │ (None, 9, 10, 256)     │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_46          │ (None, 9, 10, 256)     │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 741,796 (2.83 MB)

 Trainable params: 740,132 (2.82 MB)

 Non-trainable params: 1,664 (6.50 KB)

Epoch 1/40
    300/Unknown 43s 5ms/step - accuracy: 0.4006 - loss: 1.5833[warn] failed /content/drive/My Drive/music_dataset/Beethoven/Anhang 14-3.mid: Could not decode key with 3 flats and mode 255
300/300 ━━━━━━━━━━━━━━━━━━━━ 108s 224ms/step - accuracy: 0.4008 - loss: 1.5826 - val_accuracy: 0.2283 - val_loss: 4.3789 - learning_rate: 4.0000e-04
Epoch 2/40
291/300 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5874 - loss: 1.0623[warn] failed /content/drive/My Drive/music_dataset/Beethoven/Anhang 14-3.mid: Could not decode key with 3 flats and mode 255
300/300 ━━━━━━━━━━━━━━━━━━━━ 93s 215ms/step - accuracy: 0.5883 - loss: 1.0595 - val_accuracy: 0.2976 - val_loss: 2.3866 - learning_rate: 4.0000e-04
Epoch 3/40
297/300 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7082 - loss: 0.7900[warn] failed /content/drive/My Drive/music_dataset/Beethoven/Anhang 14-3.mid: Could not decode key with 3 flats and mode 255
300/300 ━━━━━━━━━━━━━━━━━━━━ 93s 215ms/step - accuracy: 0.7083 - loss: 0.7897 - va